In [120]:
import Functions.StringComp as SC
import Models.ImportModel as IM
import pandas as pd
import numpy as np
import json
import re

VR = IM.original()

importing from spreadsheet: VR-ansökningar 2012-2016.xlsx 

|--                  | 11%

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



|-----               | 26%

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



|--------            | 41%

...

|--------------      | 72%

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



|-----------------   | 87%

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



|--------------------| 100%



In [121]:
df = VR.df[['Dnr', 'Efternamn', 'Förnamn']]
# rename columns
df.columns = ['app_id', 'surname', 'name']

duplicate_authors = pd.DataFrame(columns=['name', 'surname', 'app_ids', 'duplicates'])


# CHECK DUPLICATES ON IDENTICAL NAME AND SURNAME

limit = None

if limit == None:
    limit = len(df)


checked_app_ids = []

for t in df.itertuples():
    if t[0] >= limit:
        break
    if t.app_id in checked_app_ids:
        continue

    subset = df[(df['name'] == t.name) & (df['surname'] == t.surname)]

    app_ids = list(subset['app_id'])
    checked_app_ids = checked_app_ids + app_ids
    app_ids_json = json.dumps(app_ids)
    duplicates = len(app_ids) - 1

    if duplicates > 0:
        temporary_df = pd.DataFrame([[t.name, t.surname, app_ids, duplicates]],
                                   columns=['name', 'surname', 'app_ids', 'duplicates'])
        duplicate_authors = duplicate_authors.append(temporary_df, ignore_index=True)

    else:
        temporary_df = pd.DataFrame([[t.name, t.surname, app_ids, duplicates]],
                               columns=['name', 'surname', 'app_ids', 'duplicates'])
        duplicate_authors = duplicate_authors.append(temporary_df, ignore_index=True)

print(duplicate_authors)


                  name             surname  \
0                Amaia  Calderón-Larrañaga   
1                Maria            Swanberg   
2      Christine-Maria              Horejs   
3                Antti             Latvala   
4              Ljubica             Perisic   
...                ...                 ...   
13694            Alena         Stsiapanava   
13695           Holger                Kohr   
13696            Dipak               Surie   
13697        Charlotta           Wallinder   
13698           Khadga               Karki   

                                                 app_ids duplicates  
0                                           [2016-00981]          0  
1      [2016-00992, 2013-03310, 2012-02563, 2013-0335...          5  
2                                           [2016-01005]          0  
3                                           [2016-01008]          0  
4                                           [2016-01016]          0  
...                        

In [122]:
writer = pd.ExcelWriter('duplicates_full.xlsx', engine='xlsxwriter')
duplicate_authors.to_excel(writer, sheet_name="duplicates", index=False)

writer.save()




In [123]:
# Import the Scrape Data
scrape5 = pd.read_excel('scrape_results_5.0.xlsm')

In [124]:

# Now we aggregate and append desired columns

def extract_cell(app_id, col):
    subset = scrape5[(scrape5['Dnr'] == app_id)]
    x = getattr(subset, col)
    isnull = x.isnull().values.any()
    if isnull:
        return []
    else:
        val = str(x.iloc[0])

        if ".0" == val[-2:]:
            val = val[:-2]

        # if there is a comma serparated number list, split it and return a list
        if re.search(r"^[\d,\.\s]+$", val) != None:
            if ", " in val:
                return val.split(", ")
            else:
                return [val]
        else:
            return [val]

print(extract_cell("2016-00981", 'Year'))
print(extract_cell("2016-00992", "Year"))

print(extract_cell("2016-01158", 'Year'))
print(extract_cell("2016-01172", "Year_unsure"))

print(extract_cell("2016-00981", 'year_of_birth'))
print(extract_cell("2016-00992", "year_of_birth"))
print(extract_cell("2016-01158", 'year_of_birth'))
print(extract_cell("2016-01172", "year_of_birth"))

print(extract_cell("2016-00981", 'Institution'))
print(extract_cell("2016-00992", "Institution"))
print(extract_cell("2016-01158", 'Institution'))
print(extract_cell("2016-01172", "Institution"))

[]
['2007']
[]
['2015', '2006']
[]
['1977']
[]
[]
[]
['Karolinska Institutet']
[]
['Karolinska Institutet']


In [125]:
def clean_list(list):

    # remove list from single-pnum lists
    clean = []
    for x in list:
        if len(x) == 1:
            # convert to integer if it looks like one
            if re.match(r"^\d+$", str(x[0])) != None:
                clean.append(int(x[0]))
            else:
                clean.append(x[0])

        elif len(x) == 0:
            clean.append(None)

        else:
            clean.append(", ".join(x))

    print("LENGTH WAS", len(list))
    print("LENGTH IS", len(clean))

    return(clean)


print(clean_list([[], ['2007'], [], ['2011'], ['2012']]))




LENGTH WAS 5
LENGTH IS 5
[None, 2007, None, 2011, 2012]


In [126]:
def aggregate_attribute(a):

    a_list = []

    for t in duplicate_authors.itertuples():
        local_list = []
        app_id_list = re.sub(r"[\[\]\']", "", str(t.app_ids)).split(", ")
        for app_id in app_id_list:
            local_list = local_list + extract_cell(app_id, a)

        #print(list(set(local_list)))

        a_list.append(list(set(local_list)))

    return clean_list(a_list)


In [127]:
libris_links = aggregate_attribute("Libris_avhandling_url")
dart_links = aggregate_attribute("Dart_avhandling_url")


LENGTH WAS 13699
LENGTH IS 13699
LENGTH WAS 13699
LENGTH IS 13699


In [128]:
print(len(libris_links))
print(len(dart_links))
# append the column
df_links = duplicate_authors.assign(libris_link=libris_links, dart_link=dart_links)

13699
13699


In [129]:
# Now we loop through and find duplicates on link level


merge_orders_libris = []
merge_orders_dart = []

checked_libris_links= []
checked_dart_links = []

# Scan libris links
for t in df_links.itertuples():

    if t.libris_link == " ":
        continue

    if t.libris_link in checked_libris_links:
        continue

    subset = df_links[(df_links['libris_link'] == t.libris_link)]

    if len(subset) > 1:
        print(len(subset))
        base_id = subset.index.values.astype(int)[0]
        dup_ids = subset.index.values.astype(int)[1:]
        print(subset[['name', 'surname']])
        print(f"Merge {dup_ids} into {base_id}")

        merge_orders_libris.append({'base': base_id, 'dup': dup_ids})
        checked_libris_links.append(subset['libris_link'].iloc[0])

# Scan dart links
for t in df_links.itertuples():

    if t.dart_link == " ":
        continue

    if t.dart_link in checked_dart_links:
        continue

    subset = df_links[(df_links['dart_link'] == t.dart_link)]

    if len(subset) > 1:
        print(len(subset))
        base_id = subset.index.values.astype(int)[0]
        dup_ids = subset.index.values.astype(int)[1:]
        print(subset[['name', 'surname']])
        print(f"Merge {dup_ids} into {base_id}")

        merge_orders_dart.append({'base': base_id, 'dup': dup_ids})
        checked_dart_links.append(subset['dart_link'].iloc[0])


2
        name          surname
25   Kotryna  Simonyté Sjödin
945  Kotryna         Simonyté
Merge [945] into 25
2
         name    surname
107    Celine  Fernandez
12749  Céline  Fernandez
Merge [12749] into 107
2
         name            surname
284    Martin           Sahlberg
12058  Martin  Häggblad Sahlberg
Merge [12058] into 284
2
            name   surname
445         Omid  Faridani
13248  Omid Reza  Faridani
Merge [13248] into 445
2
         name surname
475    jiajia    chen
12673  Jiajia    Chen
Merge [12673] into 475
2
         name             surname
543    Mikael  Brandström Durling
13359  Mikael          Brandström
Merge [13359] into 543
2
                     name surname
651   Syed Hassan Mujtaba   Jafri
1109               Hassan   Jafri
Merge [1109] into 651
2
           name   surname
660      Stefan  Hallgren
12649  Hallgren    Stefan
Merge [12649] into 660
2
        name  surname
661   Markus  Franzen
1135  Markus  Franzén
Merge [1135] into 661
2
                nam

In [130]:
print(merge_orders_libris)
print(merge_orders_dart)

merge_orders_libris_bases = [x['base'] for x in merge_orders_libris]
merge_orders_dart_bases = [x['base'] for x in merge_orders_dart]

merge_orders = []

matches = 0

for mol in merge_orders_libris:
    if mol['base'] in merge_orders_dart_bases:
        dart_index = merge_orders_dart_bases.index(mol['base'])
        dart_dup = merge_orders_dart[dart_index]['dup']
        if dart_dup == mol['dup']:
            print("MATCH", mol['dup'])
            matches = matches + 1
            merge_orders.append(mol)
        else:
            print("MISMATCH")
    else:
        print("ONLY LIBRIS", mol)
        merge_orders.append(mol)


for mod in merge_orders_dart:
    if mod['base'] not in merge_orders_libris_bases:
        print("ONLY DART", mod)
        merge_orders.append(mod)

print(len(merge_orders_libris))
print(len(merge_orders_dart))
print(matches)
print(len(merge_orders))

[{'base': 25, 'dup': array([945])}, {'base': 107, 'dup': array([12749])}, {'base': 284, 'dup': array([12058])}, {'base': 445, 'dup': array([13248])}, {'base': 475, 'dup': array([12673])}, {'base': 543, 'dup': array([13359])}, {'base': 651, 'dup': array([1109])}, {'base': 660, 'dup': array([12649])}, {'base': 661, 'dup': array([1135])}, {'base': 690, 'dup': array([8987])}, {'base': 793, 'dup': array([13535])}, {'base': 816, 'dup': array([11795])}, {'base': 882, 'dup': array([1903])}, {'base': 988, 'dup': array([1666])}, {'base': 1000, 'dup': array([11607])}, {'base': 1013, 'dup': array([9302])}, {'base': 1181, 'dup': array([1432])}, {'base': 1191, 'dup': array([11083])}, {'base': 1199, 'dup': array([1667])}, {'base': 1236, 'dup': array([11124])}, {'base': 1284, 'dup': array([1678])}, {'base': 1300, 'dup': array([1345])}, {'base': 1349, 'dup': array([1764])}, {'base': 1419, 'dup': array([1683])}, {'base': 1439, 'dup': array([1679])}, {'base': 1516, 'dup': array([2033])}, {'base': 1588, '

In [131]:
df_merging = pd.DataFrame(columns=['name', 'surname', 'app_ids', 'duplicates'])

# Now we conduct the merges according to the merger_orders
duplicate_authors_clean = duplicate_authors.copy()

for mo in merge_orders:
    keep_row = duplicate_authors_clean.iloc[mo['base']]
    print(keep_row)
    df_merging = df_merging.append(keep_row, ignore_index=True)

    for row in mo['dup']:
        dup_row = duplicate_authors_clean.iloc[row]
        print(dup_row)
        df_merging = df_merging.append(dup_row, ignore_index=True)

        for a_id in dup_row['app_ids']:
            if a_id not in keep_row['app_ids']:
                keep_row['app_ids'] =  keep_row['app_ids'] + [a_id]
            print(keep_row['app_ids'])

        duplicate_authors_clean.iloc[mo['base']] = keep_row
        #duplicate_authors.iloc[mo['base']]


print(df_merging)


name                  Kotryna
surname       Simonyté Sjödin
app_ids          [2016-01238]
duplicates                  0
Name: 25, dtype: object
name               Kotryna
surname           Simonyté
app_ids       [2012-06616]
duplicates               0
Name: 945, dtype: object
['2016-01238', '2012-06616']
name                            Celine
surname                      Fernandez
app_ids       [2016-02104, 2015-03150]
duplicates                           1
Name: 107, dtype: object
name                Céline
surname          Fernandez
app_ids       [2013-02412]
duplicates               0
Name: 12749, dtype: object
['2016-02104', '2015-03150', '2013-02412']
name                            Martin
surname                       Sahlberg
app_ids       [2016-03585, 2015-04893]
duplicates                           1
Name: 284, dtype: object
name                                        Martin
surname                          Häggblad Sahlberg
app_ids       [2014-03943, 2012-04348, 2013-05413]
d

<ipython-input-131-67ef44a83e14>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  keep_row['app_ids'] =  keep_row['app_ids'] + [a_id]


name               katinka
surname           svanberg
app_ids       [2016-00354]
duplicates               0
Name: 2033, dtype: object
['2014-06758', '2016-00354']
name                    CI
surname               SONG
app_ids       [2014-06890]
duplicates               0
Name: 1588, dtype: object
name                                Ci
surname                           Song
app_ids       [2015-06343, 2016-00598]
duplicates                           1
Name: 1811, dtype: object
['2014-06890', '2015-06343']
['2014-06890', '2015-06343', '2016-00598']
name                 Aishe
surname            Sarshad
app_ids       [2014-06900]
duplicates               0
Name: 1593, dtype: object
name                      Aishe
surname       Angeletti Sarshad
app_ids            [2015-06376]
duplicates                    0
Name: 1833, dtype: object
['2014-06900', '2015-06376']
name                  Emma
surname              Björk
app_ids       [2014-06918]
duplicates               0
Name: 1607, dtype: objec

In [132]:
print(duplicate_authors.iloc[25])
print(duplicate_authors_clean.iloc[25])

print(duplicate_authors.iloc[13446])
print(duplicate_authors_clean.iloc[13446])
print(duplicate_authors_clean.iloc[13672])

indicies_to_drop = []
print("INDICES")
for l in [x['dup'] for x in merge_orders]:
    for i in l:
        indicies_to_drop.append(i)

# Now we drop the duplicate rows that have been integrated
print(len(duplicate_authors_clean))
duplicate_authors_clean = duplicate_authors_clean.drop(indicies_to_drop)
print(len(duplicate_authors_clean))

name                  Kotryna
surname       Simonyté Sjödin
app_ids          [2016-01238]
duplicates                  0
Name: 25, dtype: object
name                           Kotryna
surname                Simonyté Sjödin
app_ids       [2016-01238, 2012-06616]
duplicates                           0
Name: 25, dtype: object
name                Carlos
surname            Vázquez
app_ids       [2014-05941]
duplicates               0
Name: 13446, dtype: object
name                            Carlos
surname                        Vázquez
app_ids       [2014-05941, 2015-05381]
duplicates                           0
Name: 13446, dtype: object
name                Carlos
surname            Vazquez
app_ids       [2015-05381]
duplicates               0
Name: 13672, dtype: object
INDICES
13699
13546


In [133]:
# And finally before we export, we update the duplicates variable
# HOWEVER NOW THE DUPLICATES VARIABLE IS OCCURANCES

#duplicate_authors_clean['duplicates']  = duplicate_authors_clean['app_ids'].len()
duplicate_authors_clean['duplicates'] = duplicate_authors_clean['app_ids'].apply(lambda x: len(x))
duplicate_authors_clean.rename(columns = {'duplicates':'occurances'}, inplace = True)

In [134]:
# Now we do the final export

writer = pd.ExcelWriter('duplicates_reduced.xlsx', engine='xlsxwriter')
duplicate_authors_clean.to_excel(writer, sheet_name="Researchers", index=False)
writer.save()


writer = pd.ExcelWriter('merged_discrepancies.xlsx', engine='xlsxwriter')
df_merging.to_excel(writer, sheet_name="duplicates", index=False)
writer.save()



In [135]:

# def clean_str(str):
#     return re.sub(r"[-]", " ", str)
#
# def do_str_comp(row, s2):
#     print(row)
#     print("THIS WAS THE SHIT")
#     return False
#
#     s1 = clean_str(s1)
#     s2 = clean_str(s2)
#     x = SC.StringComp(s1, s2, 1, 1)
#     #print(x.print_analysis())
#     if x.max_percent_words_matched > 80:
#         return True
#     else:
#         return False
#
# #do_str_comp("Martin Bjorkman", "Martin Bjan-Nykvist")


In [136]:
# def filter_df(df):
#     return df.index.str.startswith('A')


In [137]:
# limit = 2
#
# if limit == None:
#     limit = len(df)
#
#
# checked_app_ids = []

# for t in df.itertuples():
#     if t[0] >= limit:
#         break
#     if t.app_id in checked_app_ids:
#         continue
#
#     #subset = df.loc[lambda row: row['name'].str.startswith("A")]
#
#     subset = df.apply(do_str_comp,  args=(t.name, ), axis=1)
#
#
#     app_ids = list(subset['app_id'])
#     checked_app_ids = checked_app_ids + app_ids
#     app_ids_json = json.dumps(app_ids)
#     duplicates = len(app_ids) - 1
#
#     if duplicates > 0:
#        temporary_df = pd.DataFrame([[t.name, t.surname, app_ids, duplicates]],
#                                    columns=['name', 'surname', 'app_ids', 'duplicates'])
#        duplicate_authors = duplicate_authors.append(temporary_df, ignore_index=True)
#
# print(duplicate_authors)




In [138]:
# writer = pd.ExcelWriter('duplicates_full.xlsx', engine='xlsxwriter')
# duplicate_authors.to_excel(writer, sheet_name="duplicates", index=False)
#
# writer.save()